In [ ]:
!pip install -qU pip
# !pip install ipywidgets==8.1.5
# !jupyter nbextension enable --py widgetsnbextension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.3 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import torch
import random
import pydicom
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import SimpleITK as sitk
from tqdm.auto import tqdm
from functools import reduce
from collections import defaultdict
from typing import List, Dict, Tuple, Any, Optional
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

from IPython import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets.embed import embed_minimal_html


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, Subset, Sampler, WeightedRandomSampler

# data

In [ ]:
def get_patients_with_filter(dataset_path : str, target_filter: str) -> List[str]:
    """
     Finds patient IDs that have the specified MRI filter.
    """
    root          = Path(dataset_path)
    target_lower  = target_filter.casefold()   # like lower()
    patients_ok   = []

    for patient_dir in root.iterdir():                 # Path objects
        if not patient_dir.is_dir():
            continue

        dcm_dir = patient_dir / "DCM"
        if not dcm_dir.is_dir():
            continue

        if any( entry.is_dir() and target_lower in entry.name.casefold()
                for entry in dcm_dir.iterdir() ):
            patients_ok.append(patient_dir.name)

    return sorted(patients_ok)

In [ ]:
class MRILoader:
    def __init__(self, patient_ids: List[str], dataset_path: str = "patients_dcm",
                 target_filter: Optional[str] = None,
                 batch_size: int = 1,
                 reshaping_size: Optional[Tuple[int, int]] = (512, 512)):
        self.dataset_path = dataset_path
        self.batch_size = batch_size
        self.patient_ids = patient_ids
        self.max_shape = defaultdict(lambda: [0, 0, 0])  # [slices, rows, cols]
        self.reshaping_size = reshaping_size  # Может быть None

    def _load_dicom_data(self, file_path: str) -> Optional[Dict[str, Any]]:
        try:
            ds = pydicom.dcmread(file_path)
            if not hasattr(ds, 'PixelSpacing'):
                ds.PixelSpacing = [1.0, 1.0]
                print(f"PixelSpacing missing in {file_path}. Using default [1.0, 1.0].")
            if not hasattr(ds, 'SliceThickness'):
                ds.SliceThickness = 1.0
                print(f"SliceThickness missing in {file_path}. Using default 1.0.")


            if not hasattr(ds, 'pixel_array'):
                print(f"pixel data missing: {file_path}")
                return None

            # feature extraction
            return {
                'pixel_array': ds.pixel_array,
                'slice_location': getattr(ds, 'SliceLocation',
                                          ds.ImagePositionPatient[-1] if hasattr(ds, 'ImagePositionPatient') and ds.ImagePositionPatient else None),
                'image_position': getattr(ds, 'ImagePositionPatient', None),
                'pixel_spacing': tuple(getattr(ds, 'PixelSpacing', (1.0, 1.0))),
                'slice_thickness': getattr(ds, 'SliceThickness', 1.0),
                'image_orientation': getattr(ds, 'ImageOrientationPatient', None),
                'series_description': getattr(ds, 'SeriesDescription', None),
                'instance_number': getattr(ds, 'InstanceNumber', None),
                'series_time': getattr(ds, 'SeriesTime', None),
                'series_uid': getattr(ds, 'SeriesInstanceUID', None),
                'acq_number': getattr(ds, 'AcquisitionNumber', None),
                'file_path': file_path  # На всякий
            }
        except Exception as e:
            print(f"Can't read {file_path}: {e}")
            return None

    def _resize_single_slice_sitk(self,
                                 pixel_array: np.ndarray,
                                 original_spacing: Tuple[float, float],
                                 target_hw_size: Tuple[int, int],
                                 interpolator=sitk.sitkLinear) -> np.ndarray:
        """
        Resizes a single 2D slice using SimpleITK, considering its pixel spacing.
        target_hw_size: (target_Height, target_Width)
        """

        if not (original_spacing and len(original_spacing) == 2 and original_spacing[0] > 1e-9 and original_spacing[1] > 1e-9):
            print(f"Invalid original_spacing {original_spacing}. Falling back to cv2.resize.")
            return cv2.resize(pixel_array, dsize=(target_hw_size[1], target_hw_size[0]), interpolation=cv2.INTER_LANCZOS4)

        sitk_image = sitk.GetImageFromArray(pixel_array.astype(np.float32))
        # SimpleITK ожидает spacing (spacing_x, spacing_y)
        # sitk_image.SetSpacing((float(original_spacing[0]), float(original_spacing[1])))

        ### POSSIBLE BULLSHIT, BE ADVISED
        sitk_image.SetSpacing((float(original_spacing[1]), float(original_spacing[0]))) # if it comes from DICOM, it should be inverted

        original_size_wh = sitk_image.GetSize()  # (width, height)
        target_size_wh = (target_hw_size[1], target_hw_size[0])  # (target_Width, target_Height)

        if original_size_wh == target_size_wh and sitk_image.GetSpacing() == original_spacing :
             pass # Ресайз все равно нужен для приведения к целевому спейсингу вычисленному ниже

        ref_image = sitk.Image(target_size_wh, sitk_image.GetPixelIDValue())
        ref_image.SetOrigin(sitk_image.GetOrigin())
        ref_image.SetDirection(sitk_image.GetDirection())

        # PhysicalSize = VoxelSize * NumberOfVoxels
        # OriginalPhysicalSize = original_spacing * original_size_wh
        # TargetPhysicalSize = new_spacing * target_size_wh
        # OriginalPhysicalSize == TargetPhysicalSize
        # => new_spacing = original_spacing * (original_size_wh / target_size_wh)

        if target_size_wh[0] == 0 or target_size_wh[1] == 0:
            print(f"Target size for resize is zero {target_size_wh}. Using cv2.resize.")
            return cv2.resize(pixel_array, dsize=(target_hw_size[1], target_hw_size[0]), interpolation=cv2.INTER_LANCZOS4)

        new_spacing = [
            sitk_image.GetSpacing()[i] * (original_size_wh[i] / target_size_wh[i])
            for i in range(sitk_image.GetDimension())
        ]
        ref_image.SetSpacing(new_spacing)

        resampled_image = sitk.Resample(sitk_image, ref_image, sitk.Transform(),
                                        interpolator, sitk_image.GetPixelIDValue())  # sitk.sitkLinear

        return sitk.GetArrayFromImage(resampled_image)


    def load_patient_data(self, patient_id: str,
                          reshaping_size_hw: Optional[Tuple[int, int]],
                          target_filter: str) -> Dict[str, Dict[str, Any]]:
        """
        Loads patient's data for a specific series type (target_filter).
        target_filter: Exact name of the subfolder containing DICOM files for the series.
        reshaping_size_hw: Tuple (Height, Width) for resizing. If None, no resize.
        """
        patient_specific_path = os.path.join(self.dataset_path, patient_id, "DCM")
        if not os.path.isdir(patient_specific_path):
            print(f"Base DCM dir not found for {patient_id} at {patient_specific_path}")
            return {}

        series_folder_path = os.path.join(patient_specific_path, target_filter)

        if not os.path.isdir(series_folder_path):
            print(f" Series folder '{target_filter}' not found for patient {patient_id} at '{series_folder_path}'.")
            return {}

        dicom_items = []

        # Основная по SliceLocation или InstanceNumber
        dicom_files = sorted(os.listdir(series_folder_path))
        if not dicom_files:
            print(f"No DICOM files found in {series_folder_path}")
            return {}

        for name in dicom_files:
            if name.startswith('.'):
                continue

            file_path = os.path.join(series_folder_path, name)
            if os.path.isdir(file_path):
                continue

            item = self._load_dicom_data(file_path)
            if item is None:
                continue

            if item['slice_location'] is None:
                if item['instance_number'] is not None:
                    print(f"slice_location is None for {item['file_path']}. Will use instance_number for sorting key if needed.")
                else:
                    print(f"slice_location AND instance_number are None for {item['file_path']}. Cannot reliably sort. Skipping.")
                    continue  # Пропускаю срез, если нет ключа для сортировки

            dicom_items.append(item)

        if not dicom_items:
            print(f"[warn] No valid DICOM items loaded from {series_folder_path}")
            return {}

        # Группировка по SeriesInstanceUID и AcquisitionNumber
        groups = defaultdict(list)
        for item in dicom_items:
            # AcquisitionNumber может быть None
            acq_num_key = str(item['acq_number']) if item['acq_number'] is not None else "None"
            series_uid_key = str(item['series_uid']) if item['series_uid'] is not None else "UnknownUID"
            group_key = (series_uid_key, acq_num_key)
            groups[group_key].append(item)

        patient_series_data = {}  # Результат для данного target_filter

        for (series_uid, acq_number_str), series_items in groups.items():
            # Сортировка срезов внутри каждой группы
            # Приоритет: SliceLocation, затем InstanceNumber
            def sort_key_slice(x):
                sl = x['slice_location']
                inst_num = x['instance_number']
                if sl is not None:
                    try: return float(sl)
                    except (ValueError, TypeError): pass  # если не конвертится
                if inst_num is not None:
                    try: return float(inst_num)  # InstanceNumber если SliceLocation плохой
                    except (ValueError, TypeError): pass
                print(f"Cannot determine sort order for {x['file_path']}. Placing last.")
                return float('inf')  # Если оба None или невалидны, ставим в конец

            sorted_series_items = sorted(series_items, key=sort_key_slice)

            slices_np_list = []
            processed_info_list = []

            for dicom_data_item in sorted_series_items:
                pixel_arr = dicom_data_item['pixel_array']

                if pixel_arr.ndim != 2 or pixel_arr.shape[0] == 0 or pixel_arr.shape[1] == 0:
                    print(f"Invalid pixel_array shape {pixel_arr.shape} for slice in {dicom_data_item['file_path']}. Skipping.")
                    continue

                # Ресайз, если reshaping_size_hw задан
                if reshaping_size_hw is not None and len(reshaping_size_hw) == 2:
                    try:
                        resized_arr = self._resize_single_slice_sitk(
                            pixel_array=pixel_arr,
                            original_spacing=dicom_data_item['pixel_spacing'],
                            target_hw_size=reshaping_size_hw
                        )
                        slices_np_list.append(resized_arr.squeeze())
                    except Exception as e:
                        print(f"Failed to resize slice from {dicom_data_item['file_path']}: {e}. Skipping.")
                        continue
                else:  # Нет ресайза
                    slices_np_list.append(pixel_arr.squeeze())

                # Метаданные
                processed_info_list.append({
                    'slice_location': dicom_data_item['slice_location'],
                    'image_position': dicom_data_item['image_position'],
                    'slice_thickness': dicom_data_item['slice_thickness'],
                    'instance_number': dicom_data_item['instance_number'],
                    'image_orientation': dicom_data_item['image_orientation'],
                    'series_description': dicom_data_item['series_description'],
                    'pixel_spacing': dicom_data_item['pixel_spacing'],  # Это исходный спейсинг
                    'series_time': dicom_data_item['series_time'],
                    # 'file_path': dicom_data_item['file_path']  # для отладки
                })

            if not slices_np_list:
                print(f"[warn] No slices processed for group UID {series_uid}, AcqNum {acq_number_str} in {series_folder_path}")
                continue

            try:
                if reshaping_size_hw is None and len(slices_np_list) > 1:
                    first_shape = slices_np_list[0].shape
                    if not all(s.shape == first_shape for s in slices_np_list):
                        print(f"[error] Slices in group UID {series_uid}, AcqNum {acq_number_str} have inconsistent shapes and no reshaping was applied. Cannot stack. Skipping group.")
                        # В случае полного бреда информативно:
                        for i, s_arr in enumerate(slices_np_list):
                            print(f"Slice {i} shape: {s_arr.shape} from {processed_info_list[i].get('file_path', 'N/A')}")
                        continue

                tensor_3d = np.stack(slices_np_list, axis=0)  # [Z, H, W]
            except ValueError as e:
                print(f"Could not stack slices for group UID {series_uid}, AcqNum {acq_number_str} in {series_folder_path}: {e}")
                print("Slice shapes:", [s.shape for s in slices_np_list])
                continue


            # Ключ для словаря patient_series_data
            # target_filter (имя папки серии), UID и AcqNum для уникальности
            series_key = f"{target_filter}_UID-{series_uid}_ACQ-{acq_number_str}"
            patient_series_data[series_key] = {'tensor': tensor_3d, 'info_list': processed_info_list}

            # Обновление max_shape
            self.max_shape[series_key][0] = max(self.max_shape[series_key][0], tensor_3d.shape[0])
            self.max_shape[series_key][1] = max(self.max_shape[series_key][1], tensor_3d.shape[1])
            self.max_shape[series_key][2] = max(self.max_shape[series_key][2], tensor_3d.shape[2])

        return patient_series_data

In [ ]:
class PadToLargest(tio.transforms.Transform):
    def apply_transform(self, subject: tio.Subject) -> tio.Subject:
        shapes = [img.spatial_shape for img in subject.get_images()]
        target_shape = tuple(max(s[d] for s in shapes) for d in range(3))

        # Сначала синхронизируем пространство
        subject = tio.ToCanonical()(subject)
        subject = tio.Resample()(subject)

        if all(s == target_shape for s in shapes):
            return subject

        pad_or_crop = tio.CropOrPad(
            target_shape=target_shape,
            padding_mode=0,
            only_pad=True
        )
        return pad_or_crop(subject)


class MRIDataset_double(MRILoader):
    _EPS = 1e-8

    def __init__(self, patient_ids: List[str], dataset_path: str = 'patients_dcm',
                 contrast1_foldername: str = 't1_fl2d_cor',
                 contrast2_foldername: str = 't2_fl2d_cor',
                 reshaping_size_hw: Optional[Tuple[int, int]] = None, # (H, W)
                 normalize: bool = False,
                 augment: bool = False):


        self.mri_loader_instance = MRILoader(
            patient_ids=[],
            dataset_path=dataset_path,
            reshaping_size=reshaping_size_hw
        )

        self.normalize = normalize
        self.augment = augment
        self.contrast1_foldername = contrast1_foldername
        self.contrast2_foldername = contrast2_foldername
        self.reshaping_size_hw = reshaping_size_hw  # (H,W)
        self.patient_ids = patient_ids  # Список ID для этого датасета
        self.dataset_path = dataset_path  # Путь к данным
        self._build_augment_pipeline()

        self._index: List[str] = []
        self._cache: Dict[str, Dict[str, Any]] = {}

        self._n_pixels1, self._sum1, self._sum_sq1 = 0, 0.0, 0.0
        self._n_pixels2, self._sum2, self._sum_sq2 = 0, 0.0, 0.0

        for pid in self.patient_ids:

            pdata1_all_series = self.mri_loader_instance.load_patient_data(
                patient_id=pid,
                reshaping_size_hw=self.reshaping_size_hw,
                target_filter=self.contrast1_foldername
            )
            pdata2_all_series = self.mri_loader_instance.load_patient_data(
                patient_id=pid,
                reshaping_size_hw=self.reshaping_size_hw,
                target_filter=self.contrast2_foldername
            )

            if not pdata1_all_series or not pdata2_all_series:
                # print(f"Patient {pid}: missing data for one or both contrasts ('{self.contrast1_foldername}', '{self.contrast2_foldername}'). Skipping.")
                continue

            # Выбираем одну серию из pdataX_all_series по max количеству срезов
            if len(pdata1_all_series) > 1:
                # print(f"Patient {pid}, contrast 1 ({self.contrast1_foldername}): multiple series found ({list(pdata1_all_series.keys())}). Selecting one with max slices.")
                s_key1 = max(pdata1_all_series.keys(), key=lambda k: pdata1_all_series[k]['tensor'].shape[0])
            elif pdata1_all_series:
                s_key1 = next(iter(pdata1_all_series))
            else:
                continue

            if len(pdata2_all_series) > 1:
                # print(f"Patient {pid}, contrast 2 ({self.contrast2_foldername}): multiple series found ({list(pdata2_all_series.keys())}). Selecting one with max slices.")
                s_key2 = max(pdata2_all_series.keys(), key=lambda k: pdata2_all_series[k]['tensor'].shape[0])
            elif pdata2_all_series:
                s_key2 = next(iter(pdata2_all_series))
            else:
                continue

            self._cache[pid] = {
                'img1_data': pdata1_all_series[s_key1],
                'img2_data': pdata2_all_series[s_key2]
            }
            self._index.append(pid)

            # Статистика для нормализации
            if self.normalize:
                arr1 = pdata1_all_series[s_key1]['tensor'].astype(np.float32)
                arr2 = pdata2_all_series[s_key2]['tensor'].astype(np.float32)

                self._n_pixels1 += arr1.size
                self._sum1 += arr1.sum()
                self._sum_sq1 += np.square(arr1).sum()

                self._n_pixels2 += arr2.size
                self._sum2 += arr2.sum()
                self._sum_sq2 += np.square(arr2).sum()

        # Параметры нормализации
        if self.normalize and self._n_pixels1 > 0 and self._n_pixels2 > 0:
            self._mean1 = self._sum1 / self._n_pixels1
            var1 = self._sum_sq1 / self._n_pixels1 - self._mean1**2
            self._std1 = float(np.sqrt(max(var1, self._EPS)))

            self._mean2 = self._sum2 / self._n_pixels2
            var2 = self._sum_sq2 / self._n_pixels2 - self._mean2**2
            self._std2 = float(np.sqrt(max(var2, self._EPS)))
            print(f"Normalization stats for {self.contrast1_foldername}: mean={self._mean1:.4f}, std={self._std1:.4f}")
            print(f"Normalization stats for {self.contrast2_foldername}: mean={self._mean2:.4f}, std={self._std2:.4f}")
        else:
            # Дефолтные значения
            self._mean1, self._std1 = 0.0, 1.0
            self._mean2, self._std2 = 0.0, 1.0


    @staticmethod
    def _slice_meta_to_vec(m: Dict[str, Any]) -> np.ndarray:
        vec = []
        vec.append(float(m.get('slice_location', 0.) or 0.))
        vec.extend(map(float, m.get('image_position', [0., 0., 0.]) or [0.,0.,0.]))
        vec.append(float(m.get('slice_thickness', 0.) or 0.))
        vec.append(float(m.get('instance_number', 0) or 0))
        vec.extend(map(float, m.get('image_orientation', [0.]*6) or [0.]*6))
        ps = m.get('pixel_spacing', (0.,0.))
        if ps is None or len(ps) < 2: ps = (0.,0.)
        vec.extend(map(float, ps[:2]))
        return np.asarray(vec, dtype=np.float32)

    @classmethod
    def metas_to_tensor(cls, info_list: List[Dict[str, Any]]) -> np.ndarray:
        if not info_list:
            return np.empty((0, 14), dtype=np.float32)
        return np.stack([cls._slice_meta_to_vec(m) for m in info_list])

    def set_normalization(self, mean1: float, std1: float, mean2: float, std2: float):
        self.normalize = True
        self._mean1, self._std1 = float(mean1), float(std1)
        self._mean2, self._std2 = float(mean2), float(std2)
        print(f"Normalization stats externally set for {self.contrast1_foldername}: mean={self._mean1:.4f}, std={self._std1:.4f}")
        print(f"Normalization stats externally set for {self.contrast2_foldername}: mean={self._mean2:.4f}, std={self._std2:.4f}")


    def get_max_slices(self) -> int:
        if not self._index: return 0
        max_z = 0
        for pid in self._index:

            img1_tensor = self._cache[pid]['img1_data']['tensor']
            img2_tensor = self._cache[pid]['img2_data']['tensor']
            max_z = max(max_z, img1_tensor.shape[0], img2_tensor.shape[0])
        return max_z


    # def _build_augment_pipeline(self):
    #     if not self.augment:
    #         self.transforms_geom = None
    #         self.transforms_intensity = None
    #         return

    #     self.geom_transforms_config = [
    #         (tio.RandomFlip, {'axes': ('LR',), 'p': 0.3}),
    #         (tio.RandomAffine, {'scales': (0.85, 1.15), 'degrees': 15, 'p': 0.5}),
    #         (tio.RandomElasticDeformation, {'num_control_points': 7, 'max_displacement': 3, 'p': 0.3}),
    #         (tio.OneOf, {'transforms': [
    #             tio.RandomMotion(degrees=10, translation=10),
    #             tio.RandomGhosting(intensity=0.5),
    #             tio.RandomSpike(intensity=0.5)
    #         ], 'p': 0.3})
    #     ]

    #     self.intensity_transforms_config = [
    #         (tio.RandomBiasField, {'p': 0.4}),
    #         (tio.RandomNoise, {'p': 0.4, 'mean': 0.0, 'std': 0.05}),
    #         (tio.RandomGamma, {'p': 0.4, 'log_gamma': (-0.3, 0.3)}),
    #         (tio.RandomBlur, {'p': 0.2, 'std': (0.5, 1.5)}),
    #         (tio.OneOf, {'transforms': [
    #             tio.RandomMotion(),
    #             tio.RandomGhosting(),
    #             tio.RandomSpike()
    #         ], 'p': 0.2})
    #     ]

    def _build_augment_pipeline(self):
        if not self.augment:
            self.transforms_geom = None
            self.transforms_intensity1 = None
            self.transforms_intensity2 = None
            return

        # Геометрические - одинаково к обоим контрастам
        self.geom_transforms_config = [
            (tio.RandomFlip, {'axes': ('LR',), 'p': 0.5}),
            (tio.RandomAffine, {'scales': (0.9, 1.1), 'degrees': 10, 'p': 0.7}),
            (tio.RandomElasticDeformation, {'num_control_points': 5, 'max_displacement': 4.5, 'p': 0.3})
        ]

        # Интенсивностные для первого контраста (T1)
        self.intensity_transforms_config1 = [
            (tio.RandomBiasField, {'p': 0.3, 'coefficients': 0.5}),  # Меньше для T1
            (tio.RandomNoise, {'p': 0.3, 'mean': 0.0, 'std': 0.03}),
            (tio.RandomGamma, {'p': 0.3, 'log_gamma': (-0.2, 0.2)}),  # Меньший диапазон
            (tio.RandomBlur, {'p': 0.2, 'std': (0.5, 1.0)})
        ]

        # Интенсивностные для второго контраста (T2/FLAIR)
        self.intensity_transforms_config2 = [
            (tio.RandomBiasField, {'p': 0.4, 'coefficients': 0.7}),  # Сильнее для T2/FLAIR
            (tio.RandomNoise, {'p': 0.4, 'mean': 0.0, 'std': 0.04}),
            (tio.RandomGamma, {'p': 0.4, 'log_gamma': (-0.3, 0.3)}),  # Больший диапазон
            (tio.RandomBlur, {'p': 0.2, 'std': (0.5, 1.5)})
        ]

    def __len__(self) -> int:
        return len(self._index)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        pid = self._index[idx]
        item1_data = self._cache[pid]['img1_data']
        item2_data = self._cache[pid]['img2_data']

        img1 = item1_data["tensor"]
        img2 = item2_data["tensor"]

        # Нормализация
        if self.normalize:
            img1 = (img1.astype(np.float32) - self._mean1) / (self._std1 + self._EPS)
            img2 = (img2.astype(np.float32) - self._mean2) / (self._std2 + self._EPS)
        else:
            img1 = img1.astype(np.float32)
            img2 = img2.astype(np.float32)

        if self.augment:
            processed_img1 = torch.from_numpy(img1[None, ...])
            processed_img2 = torch.from_numpy(img2[None, ...])

            for TransformClass, kwargs in self.geom_transforms_config:
                current_p = kwargs.get('p', 1.0)
                if random.random() < current_p:
                    transform_kwargs_no_p = {k:v for k,v in kwargs.items() if k != 'p'}
                    transform_instance = TransformClass(**transform_kwargs_no_p)

                    # Один и тот же трансформер для обоих изображений
                    s_img1 = tio.ScalarImage(tensor=processed_img1.clone(), affine=np.eye(4))
                    s_img2 = tio.ScalarImage(tensor=processed_img2.clone(), affine=np.eye(4))

                    # Одинаковая трансформация
                    out_subj1 = transform_instance(tio.Subject(image=s_img1))
                    out_subj2 = transform_instance(tio.Subject(image=s_img2))

                    processed_img1 = out_subj1.image.data
                    processed_img2 = out_subj2.image.data

            # Для первого контраста
            for TransformClass, kwargs in self.intensity_transforms_config1:
                current_p = kwargs.get('p', 1.0)
                if random.random() < current_p:
                    transform_kwargs_no_p = {k:v for k,v in kwargs.items() if k != 'p'}
                    transform_instance = TransformClass(**transform_kwargs_no_p)
                    s_img1 = tio.ScalarImage(tensor=processed_img1.clone())
                    out_subj1 = transform_instance(tio.Subject(image=s_img1))
                    processed_img1 = out_subj1.image.data

            # Для второго контраста
            for TransformClass, kwargs in self.intensity_transforms_config2:
                current_p = kwargs.get('p', 1.0)
                if random.random() < current_p:
                    transform_kwargs_no_p = {k:v for k,v in kwargs.items() if k != 'p'}
                    transform_instance = TransformClass(**transform_kwargs_no_p)
                    s_img2 = tio.ScalarImage(tensor=processed_img2.clone())
                    out_subj2 = transform_instance(tio.Subject(image=s_img2))
                    processed_img2 = out_subj2.image.data

            img1 = processed_img1.squeeze(0).numpy()
            img2 = processed_img2.squeeze(0).numpy()

        return {
            "img1": img1,
            "img2": img2,
            "meta1": self.metas_to_tensor(item1_data["info_list"]),
            "meta2": self.metas_to_tensor(item2_data["info_list"]),
            "patient_id": pid,
        }

In [ ]:
def collate_fn_double(
    batch: list,
    fixed_z: Optional[int] = None,
    fixed_h: Optional[int] = None,
    fixed_w: Optional[int] = None,
    pad_value1: float = 0.0,
    pad_value2: float = 0.0
) -> dict:
    if not batch:
        return {}

    if fixed_z is not None: max_z = fixed_z
    else: max_z = max(max(b["img1"].shape[0], b["img2"].shape[0]) for b in batch) if batch else 0

    if fixed_h is not None: max_h = fixed_h
    else: max_h = max(max(b["img1"].shape[1], b["img2"].shape[1]) for b in batch) if batch else 0

    if fixed_w is not None: max_w = fixed_w
    else: max_w = max(max(b["img1"].shape[2], b["img2"].shape[2]) for b in batch) if batch else 0

    # Отладочное
    # if max_z == 0 or max_h == 0 or max_w == 0 and any(b["img1"].size > 0 for b in batch): # Проверяем, что есть непустые картинки
    #     if any(b["img1"].size > 0 for b in batch): # Если были непустые изображения, но max_h/w стали 0
    #          print(f"collate_fn_double: max_h or max_w is 0 with non-empty batch items.")


    def _collate_pad_to_shape(arr, shape, pad_value=0.0):
        if arr.shape[0] > shape[0] or arr.shape[1] > shape[1] or arr.shape[2] > shape[2]:
            copy_z = min(arr.shape[0], shape[0])
            copy_h = min(arr.shape[1], shape[1])
            copy_w = min(arr.shape[2], shape[2])

            out = np.full(shape, pad_value, dtype=arr.dtype)
            out[:copy_z, :copy_h, :copy_w] = arr[:copy_z, :copy_h, :copy_w]
            return out

        out = np.full(shape, pad_value, dtype=arr.dtype)
        z, h, w = arr.shape
        out[:z, :h, :w] = arr
        return out

    def _collate_pad_meta(arr, z_len):
        if arr.ndim == 1:
            if arr.size == 0 and z_len > 0:
                 return np.zeros((z_len, 14), dtype=np.float32)  # 14 - число мета-фич
            # print(f"_collate_pad_meta: meta array is 1D with shape {arr.shape}.")

        num_meta_features = arr.shape[1] if arr.ndim == 2 and arr.shape[1] > 0 else 14 # 14 - дефолт
        if arr.size == 0 and z_len > 0:  # Если массив пустой, а z_len > 0
             return np.zeros((z_len, num_meta_features), dtype=np.float32)

        out = np.zeros((z_len, num_meta_features), dtype=arr.dtype)
        copy_z = min(arr.shape[0], z_len)
        if arr.size > 0:
            out[:copy_z] = arr[:copy_z]
        return out

    images1 = [_collate_pad_to_shape(b["img1"], (max_z, max_h, max_w), pad_value1)[None] for b in batch]
    images2 = [_collate_pad_to_shape(b["img2"], (max_z, max_h, max_w), pad_value2)[None] for b in batch]
    # metas1  = [_collate_pad_meta(b["meta1"], max_z) for b in batch]
    # metas2  = [_collate_pad_meta(b["meta2"], max_z) for b in batch]

    mask1 = [np.concatenate([np.ones(b["img1"].shape[0]), np.zeros(max_z - b["img1"].shape[0])]).astype(np.uint8) for b in batch]
    mask2 = [np.concatenate([np.ones(b["img2"].shape[0]), np.zeros(max_z - b["img2"].shape[0])]).astype(np.uint8) for b in batch]

    return {
        "img1": np.stack(images1),
        "img2": np.stack(images2),
        # "meta1": np.stack(metas1),
        # "meta2": np.stack(metas2),
        "mask1": np.stack(mask1),
        "mask2": np.stack(mask2),
        "patient_ids": [b["patient_id"] for b in batch],
    }


class MRIDataloader_double:
    def __init__(self,
                 dataset: MRIDataset_double,
                 batch_size: int = 2,
                 shuffle: bool = False,
                 drop_last: bool = False,
                 sampler: Optional[Sampler] = None,  # Sampler из torch.utils.data.sampler
                 fixed_z: Optional[int] = None,  # Для collate_fn
                 fixed_h: Optional[int] = None,  # Для collate_fn
                 fixed_w: Optional[int] = None   # Для collate_fn
                ):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle and sampler is None
        self.drop_last = drop_last
        self.sampler = sampler
        self.fixed_z = fixed_z
        self.fixed_h = fixed_h
        self.fixed_w = fixed_w

    def __iter__(self):
        if self.sampler is not None:
            idxs = list(self.sampler)
        else:
            idxs = list(range(len(self.dataset)))
            if self.shuffle:
                random.shuffle(idxs)

        pad_val1, pad_val2 = 0.0, 0.0
        if self.dataset.normalize:
            std1_eff = self.dataset._std1 + self.dataset._EPS
            std2_eff = self.dataset._std2 + self.dataset._EPS
            if std1_eff < self.dataset._EPS:
                std1_eff = 1.0
            if std2_eff < self.dataset._EPS:
                std2_eff = 1.0

            pad_val1 = (0.0 - self.dataset._mean1) / std1_eff
            pad_val2 = (0.0 - self.dataset._mean2) / std2_eff


        batch_items = []
        for i in idxs:
            batch_items.append(self.dataset[i])
            if len(batch_items) == self.batch_size:
                yield collate_fn_double(
                    batch_items,
                    fixed_z=self.fixed_z, fixed_h=self.fixed_h, fixed_w=self.fixed_w,
                    pad_value1=pad_val1, pad_value2=pad_val2
                )
                batch_items = []
        if batch_items and not self.drop_last:
            yield collate_fn_double(
                batch_items,
                fixed_z=self.fixed_z, fixed_h=self.fixed_h, fixed_w=self.fixed_w,
                pad_value1=pad_val1, pad_value2=pad_val2
            )

    def __len__(self):
        if self.sampler is not None:
            num_samples = len(self.sampler)
        else:
            num_samples = len(self.dataset)

        if self.drop_last:
            return num_samples // self.batch_size
        else:
            return (num_samples + self.batch_size - 1) // self.batch_size

In [ ]:
df_targets = pd.read_csv('/kaggle/input/patients-mri/Hands_mapped_targets_only_4.csv', encoding="cp1251").drop(columns="Column1")

conditions = [
    df_targets["Гистология"] == "Глиобластома",
    df_targets["Гистология"].str.contains("Анапластическая", na=False),
    df_targets["Гистология"].str.contains("G3", na=False),
    df_targets["Гистология"].str.contains("Диффузная", na=False),
    df_targets["Гистология"].str.contains("G2", na=False),
]

choices = [4, 3, 3, 2, 2]

df_targets["grade"] = np.select(conditions, choices, default=1)
df_targets["1p/19q"] = df_targets["1p/19q"].replace({"_": np.nan})
df_targets.loc[df_targets[(df_targets.Гистология.str.lower().str.contains("астроцитома")) & (df_targets["1p/19q"] == 1)].index, "Гистология"] = "Анапластическая олигодендроглиома G3"
notnull = df_targets[(~df_targets[["1p/19q", "MGMT", "IDH1"]].isnull().any(axis=1))\
    & (df_targets.TargetDirectory.notna())\
    & (df_targets.grade.ne(1))].TargetDirectory.values

df_targets[["1p/19q", "MGMT", "IDH1"]] = df_targets[["1p/19q", "MGMT", "IDH1"]].astype(float)

df_targets

,1p/19q,FLAIR до,IDH1,IDH2,MGMT,TargetDirectory,Биопсия до?,Возр-т,Гистология,Дифф-я до,МРТ до 3Тл,Повтор-ная?,Т1 до,Т1 с к до,Т2 до,Только биопсия?,Трактогра-фия до,м-0 ж-1,grade
0,0.0,0.0,0.0,0.0,0.0,patient1,0.0,66.0,Глиобластома,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,4
1,1.0,1.0,1.0,0.0,0.0,patient2,0.0,31.0,Анапластическая олигодендроглиома G3,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,3
2,0.0,1.0,1.0,0.0,1.0,patient3,0.0,24.0,Глиобластома,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,4
3,0.0,1.0,0.0,0.0,0.0,patient4,0.0,69.0,Глиобластома,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,4
4,0.0,1.0,0.0,0.0,1.0,patient5,0.0,61.0,Анапластическая олигодендроглиома G3,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,0.0,1.0,0.0,0.0,0.0,patient162,0.0,44.0,Глиобластома,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,4
168,0.0,0.0,0.0,0.0,0.0,patient163,0.0,36.0,Глиобластома,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,4
169,0.0,1.0,1.0,0.0,1.0,patient164,0.0,23.0,Глиобластома,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,4
170,0.0,1.0,0.0,0.0,0.0,NaN,0.0,59.0,Глиобластома,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,4


In [ ]:
def compute_class_weights(df_targets, target_names):
    bin_alphas = {}
    for target in target_names:
        if target == 'grade':
            continue
        counts = df_targets[target].value_counts().to_dict()
        n_pos = counts.get(1, 0)
        n_neg = counts.get(0, 0)
        total = n_pos + n_neg + 1e-6
        alpha = n_neg / total
        bin_alphas[target] = alpha

    # Для grade
    grade_counts  = df_targets['grade'].value_counts().sort_index().values
    beta = 0.999
    eff_num = (1 - beta**grade_counts) / (1 - beta)
    grade_weights = torch.tensor(eff_num.sum() / eff_num, dtype=torch.float)

    return bin_alphas, grade_weights

# model

In [ ]:
# USE ONLY IF NEEDED

if_needed=False

def seed_everything(seed: int = 1337):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed); torch.backends.cudnn.deterministic = True
if if_needed:
    seed_everything()

In [ ]:
class ChannelAttention3D(nn.Module):
    def __init__(self, ch, reduction=16):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.AdaptiveAvgPool3d(1),
            nn.Conv3d(ch, ch // reduction, 1, bias=False),
            nn.GELU(),
            nn.Conv3d(ch // reduction, ch, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(self.mlp(x))

class SpatialAttention3D(nn.Module):
    def __init__(self, kernel=7):
        super().__init__()
        padding = kernel // 2
        self.conv = nn.Conv3d(2, 1, kernel, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg = torch.mean(x, 1, keepdim=True)
        mx , _ = torch.max(x , 1, keepdim=True)
        attn = self.sigmoid(self.conv(torch.cat([avg, mx], 1)))
        return x * attn

class CBAM3D(nn.Module):
    def __init__(self, ch, reduction=16, kernel=7):
        super().__init__()
        self.ca = ChannelAttention3D(ch, reduction)
        self.sa = SpatialAttention3D(kernel)

    def forward(self, x):
        return self.sa(self.ca(x))

def gn(ch, groups=8):
    g = min(groups, ch)
    while ch % g: g -= 1
    return nn.GroupNorm(g, ch)

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch, mid_ch=None):
        super().__init__()
        mid = mid_ch or out_ch
        self.net = nn.Sequential(
            nn.Conv3d(in_ch, mid, 3, padding=1, bias=False),
            gn(mid), nn.GELU(),
            nn.Conv3d(mid, out_ch, 3, padding=1, bias=False),
            gn(out_ch), nn.GELU())
    def forward(self, x): return self.net(x)

class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, 3, stride=2, padding=1, bias=False),
            gn(out_ch), nn.GELU(),
            DoubleConv(out_ch, out_ch))
    def forward(self, x): return self.net(x)

class Up(nn.Module):
    def __init__(self, in_ch, skip_ch, out_ch, bilinear=True):
        super().__init__()
        self.up   = nn.Upsample(scale_factor=2, mode='trilinear',
                                align_corners=False) if bilinear else \
                    nn.ConvTranspose3d(in_ch, in_ch, 2, stride=2)
        self.conv = DoubleConv(in_ch + skip_ch, out_ch, in_ch // 2)

    def _pad(self, x, ref):
        dz,dy,dx = [ref.size(i)-x.size(i) for i in (2,3,4)]
        return F.pad(x,[dx//2,dx-dx//2,
                        dy//2,dy-dy//2,
                        dz//2,dz-dz//2])

    def forward(self, x, skip):
        x = self.up(x)
        x = self._pad(x, skip)
        x = torch.cat([skip, x], 1)
        return self.conv(x)

class CBAM3D(nn.Module):
    def __init__(self, ch, reduction=16, kernel=7):
        super().__init__()
        self.ca = ChannelAttention3D(ch, reduction)
        self.sa = SpatialAttention3D(kernel)

    def forward(self, x):
        return self.sa(self.ca(x))


class LiteUNet3D_D(nn.Module):
    def __init__(self, in_ch=1, base_ch=8, bilinear=True):
        super().__init__()
        c = base_ch

        self.inc   = DoubleConv(in_ch, c)
        self.down1 = Down(c,   c*2)
        self.down2 = Down(c*2, c*4)
        self.down3 = Down(c*4, c*8)
        self.down4 = Down(c*8, c*8)

        self.bottleneck = nn.Sequential(
            DoubleConv(c*8, c*8),
            DoubleConv(c*8, c*8))
        self.cbam = CBAM3D(c*8)

        self.up3  = Up(c*8, c*8, c*8, bilinear)    # Z/8
        self.up2  = Up(c*8, c*4, c*4, bilinear)    # Z/4
        self.up1  = Up(c*4, c*2, c*2, bilinear)    # Z/2

        self.gap = nn.AdaptiveAvgPool3d(1)
        self.out_features = c*2  # 16 при base=8

    def forward(self, x):
        x0 = self.inc(x)
        x1 = self.down1(x0)
        x2 = self.down2(x1)
        x3 = self.down3(x2)
        x4 = self.down4(x3)

        x4 = self.cbam(self.bottleneck(x4))

        x = self.up3(x4, x3)
        x = self.up2(x,  x2)
        x = self.up1(x,  x1)

        return self.gap(x).flatten(1)

In [ ]:
class ClassificationHead(nn.Module):
    def __init__(self, in_dim, hidden_factor=2,
                 num_classes=1, p_drop=0.1, drop_path=0.1):
        super().__init__()
        hidden = in_dim * hidden_factor
        self.fc1 = nn.Linear(in_dim, hidden)
        self.act = nn.GELU()
        self.drop = nn.Dropout(p_drop)
        self.fc2 = nn.Linear(hidden, num_classes)
        self.drop_path = drop_path

    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.drop(x)
        return self.fc2(x)

class GradeHead(nn.Module):
    def __init__(self, in_dim, hidden_factor=4, num_classes=3, dropout=0.3):
        super().__init__()
        hidden = in_dim * hidden_factor
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.LayerNorm(hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden // 2),
            nn.LayerNorm(hidden // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden // 2, num_classes)
        )

        # Инициализация с предвзятостью к равномерному распределению
        with torch.no_grad():
            last_layer = self.net[-1]
            if isinstance(last_layer, nn.Linear):
                last_layer.weight.data.normal_(0, 0.01)
                last_layer.bias.data.zero_()

    def forward(self, x):
        return self.net(x)

In [ ]:
class SimpleUNetClassifier(nn.Module):
    def __init__(self, target_names, in_channels=1, base_channels=8, feat_dim=64, num_binary=3):
        super().__init__()
        # Два отдельных UNet
        self.unet1 = LiteUNet3D_D(in_channels, base_channels)
        self.unet2 = LiteUNet3D_D(in_channels, base_channels)
        unet_feat = self.unet1.out_features  # 16

        # Объединяем признаки
        self.feat_proj = nn.Sequential(
            nn.Linear(unet_feat, feat_dim),
            nn.GELU(),
            nn.LayerNorm(feat_dim))

        self.target_names = target_names
        self.heads = nn.ModuleList(
            [ClassificationHead(feat_dim, num_classes=1) for _ in range(num_binary)] +
            [ClassificationHead(feat_dim, num_classes=3)]
        )

    def forward(self, img1, img2):
        feat1 = self.unet1(img1)  # (B, 16)
        feat2 = self.unet2(img2)  # (B, 16)
        # feat = torch.sum(torch.stack([feat1, feat2]), dim=0)  # (B, 16)
        feat = torch.mean(torch.stack([feat1, feat2]), dim=0) # (B, 16)
        # feat = torch.cat([feat1, feat2], dim=1)  # (B, 32)
        feat = self.feat_proj(feat)              # (B, 64)
        return {n: h(feat) for n, h in zip(self.target_names, self.heads)}

# training

In [ ]:
def run_epoch(loader, model, criterion_dict, optimizer=None, accum_steps=1,
              train=True, device='cpu',
              global_step=0, warmup_steps=0, base_lr=1e-3, initial_lr_factor=0.01,
              df_targets=None):
    is_train = train and optimizer is not None
    model.train() if is_train else model.eval()
    torch.set_grad_enabled(is_train)

    total_loss = 0.
    n_samples = 0
    all_targets = defaultdict(list)
    all_preds = defaultdict(list)
    correct = defaultdict(int)

    pbar = tqdm(loader, desc='train' if is_train else 'valid', leave=False)
    if is_train:
        optimizer.zero_grad()
    optimizer_steps = 0

    for step, batch in enumerate(pbar):
        imgs = torch.from_numpy(batch['img1'].astype(np.float32)).to(device)
        imgs2 = torch.from_numpy(batch['img2'].astype(np.float32)).to(device)
        ids = batch['patient_ids']
        bs = imgs.size(0)
        # --- Таргеты ---
        batch_df = df_targets[df_targets['TargetDirectory'].isin(ids)].set_index('TargetDirectory').loc[ids].reset_index()
        targets = {}
        for name in model.target_names:
            if name == 'grade':
                targets[name] = torch.tensor(batch_df['grade'].values - 2, dtype=torch.long).to(device)
            else:
                targets[name] = torch.from_numpy(batch_df[name].values.astype(np.float32)).view(-1, 1).to(device)

        # Forward
        outputs = model(imgs, imgs2)
        loss = 0.0
        for name in model.target_names:
            loss += criterion_dict[name](outputs[name], targets[name])

        # Backward
        if is_train:
            (loss / accum_steps).backward()
            if (step + 1) % accum_steps == 0 or (step + 1) == len(loader):
                # Warmup LR
                if global_step + optimizer_steps < warmup_steps:
                    lr_scale = (global_step + optimizer_steps + 1) / warmup_steps
                    start_lr = base_lr * initial_lr_factor
                    end_lr = base_lr
                    current_lr = start_lr + (end_lr - start_lr) * lr_scale
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = current_lr
                optimizer.step()
                optimizer.zero_grad()
                optimizer_steps += 1

        # Metrics
        n_samples += bs
        total_loss += loss.item() * bs

        with torch.no_grad():
            for name in model.target_names:
                out = outputs[name]
                tgt = targets[name]
                if name == 'grade':
                    pred = torch.argmax(out, dim=1)
                else:
                    pred = (torch.sigmoid(out) > 0.5).float()
                all_preds[name].append(pred.cpu())
                all_targets[name].append(tgt.cpu())
                correct[name] += (pred == tgt).sum().item()

            # Logging
            avg_loss = total_loss / n_samples
            postfix = {'loss': f'{avg_loss:.4f}'}
            for name in model.target_names:
                acc = correct[name] / n_samples
                postfix[f'{name}_acc'] = f'{acc:.3f}'
            pbar.set_postfix(postfix)

    # metrics calc
    avg_loss_epoch = total_loss / n_samples
    avg_f1_scores = {}
    print(f"\nEpoch {pbar.desc} finished. Calculating final metrics...")
    for name in model.target_names:
        y_true = torch.cat(all_targets[name]).numpy()
        y_pred = torch.cat(all_preds[name]).numpy()
        if name != 'grade':
            y_true = y_true.squeeze()
            y_pred = y_pred.squeeze()
        f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
        acc = accuracy_score(y_true, y_pred)
        avg_f1_scores[f'{name}_macro_f1'] = f1
        print(f"--- {name} ---")
        print(f"  Macro F1: {f1:.4f}")
        print(f"  Accuracy: {acc:.4f}")
        print(f"  Targets: {np.bincount(y_true.astype(int))}")
        print(f"  Preds:   {np.bincount(y_pred.astype(int))}")
        print("-" * 20)

    updated_global_step = global_step + optimizer_steps if is_train else global_step
    return avg_loss_epoch, avg_f1_scores, updated_global_step

In [ ]:
all_targets = ["IDH1", "MGMT", "1p/19q", "grade"]
all_targets

['IDH1', 'MGMT', '1p/19q', 'grade']

In [ ]:
%%time
dataset_directory = "/kaggle/input/patients-mri/patients_dcm"
target_filter = "t1_fl2d_cor"

patients_with_filter = get_patients_with_filter(dataset_directory, target_filter)
print(len(patients_with_filter))


dataset_directory = "/kaggle/input/patients-mri/patients_dcm"
target_filter = "t1_fl2d_cor"

patients_with_filter2 = get_patients_with_filter(dataset_directory, "t2_tirm_cor_dark-fluid_3mm")
print(len(patients_with_filter2))

147
139
CPU times: user 139 ms, sys: 271 ms, total: 410 ms
Wall time: 21.5 s


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, binary=False, multi_label_smoothing=0.0):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.binary = binary
        self.multi_label_smoothing = multi_label_smoothing

    def forward(self, inputs, targets):
        if self.binary:
            # Для бинарной
            bce_loss = F.binary_cross_entropy_with_logits(inputs, targets)

            probs = torch.sigmoid(inputs)
            pt = torch.where(targets == 1, probs, 1 - probs)

            # Focal loss
            focal_weight = (1 - pt) ** self.gamma

            if self.alpha is not None:
                alpha_t = torch.where(targets == 1, self.alpha, 1 - self.alpha)
                focal_weight = alpha_t * focal_weight

            loss = focal_weight * bce_loss
        else:
            # Для грейда
            if self.alpha is not None:
                ce_loss = F.cross_entropy(inputs, targets, weight=self.alpha, label_smoothing=self.multi_label_smoothing)
            else:
                ce_loss = F.cross_entropy(inputs, targets, label_smoothing=self.multi_label_smoothing)

            pt = torch.exp(-ce_loss)
            loss = (1 - pt) ** self.gamma * ce_loss

        return loss

In [ ]:
def compute_inverse_freq_weights(labels, num_classes=None, epsilon=1e-6):
    if num_classes is None:
        num_classes = np.max(labels) + 1
    class_counts = np.bincount(labels.astype(int), minlength=num_classes)
    inv_freq = 1.0 / (class_counts + epsilon)
    inv_freq = inv_freq / np.sum(inv_freq)
    return inv_freq

def make_multi_target_sampler(ids, df, target_cols, oversample_factor=1.0, epsilon=1e-6):
    df_fold = df.set_index('TargetDirectory').loc[ids]
    all_weights = []
    for col, num_classes in target_cols.items():
        labels = df_fold[col].values
        if col == 'grade':
            labels = labels - 2
        inv_freq = compute_inverse_freq_weights(labels, num_classes, epsilon)
        all_weights.append(inv_freq[labels.astype(int)])
    sample_weights = np.max(np.stack(all_weights, axis=1), axis=1)  # Maxxing for all the tasks now, possible to take mean to rebalance
    num_samples = int(len(ids) * oversample_factor)
    return WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

In [ ]:
N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

united = list(set(patients_with_filter) & set(patients_with_filter2) & set(notnull))
len(united)

strat_labels = []
for pid in united:
    g = int(df_targets.query("TargetDirectory == @pid")['grade'].iloc[0])
    strat_labels.append(g)

In [ ]:
N_EPOCHS = 30
ACCUM_STEPS = 16
BASE_LR = 3e-4
WARMUP_EPOCHS = 5
INITIAL_LR_FACTOR = 1e-2
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

target_columns_config = {'grade': 3, 'IDH1': 2, 'MGMT': 2, '1p/19q': 2}
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(united, strat_labels), 1):
    print(f'\n==========  FOLD {fold}/{N_FOLDS}  ({len(train_idx)} train / {len(val_idx)} val) ==========')
    train_ids = [united[i] for i in train_idx]
    val_ids = [united[i] for i in val_idx]

    # Losses
    bin_alphas, grade_weights = compute_class_weights(df_targets[df_targets.TargetDirectory.isin(train_ids)], all_targets)
    criterion_dict = {}
    for name in all_targets:
        if name == 'grade':
            criterion_dict[name] = FocalLoss(gamma=2, alpha=grade_weights.to(DEVICE), binary=False)
        else:
            alpha = bin_alphas.get(name, 0.5)
            if isinstance(alpha, (list, tuple, np.ndarray)):
                alpha = alpha[1] if len(alpha) == 2 else 0.5
            criterion_dict[name] = FocalLoss(gamma=2, alpha=torch.tensor(alpha, device=DEVICE), binary=True)

    # Datasets
    train_ds = MRIDataset_double(train_ids, dataset_path="/kaggle/input/patients-mri/patients_dcm", contrast1='t1_fl2d_cor', contrast2="t2_tirm_cor_dark-fluid_3mm",
                                 reshaping_size=(256,256), normalize=True)
    val_ds = MRIDataset_double(val_ids, dataset_path="/kaggle/input/patients-mri/patients_dcm", contrast1='t1_fl2d_cor', contrast2="t2_tirm_cor_dark-fluid_3mm",
                               reshaping_size=(256,256), normalize=False)
    val_ds.set_normalization(train_ds._mean1, train_ds._std1, train_ds._mean2, train_ds._std2)

    # Sampler and loaders
    train_sampler = make_multi_target_sampler(train_ids, df_targets, target_columns_config, oversample_factor=2)
    train_loader = MRIDataloader_double(train_ds, batch_size=1, shuffle=False, sampler=train_sampler)
    val_loader = MRIDataloader_double(val_ds, batch_size=1, shuffle=False)

    # Warmup steps
    num_train_steps_per_epoch = len(train_loader) // ACCUM_STEPS
    WARMUP_STEPS = WARMUP_EPOCHS * num_train_steps_per_epoch
    print(f"Warmup: {WARMUP_EPOCHS} epochs, {WARMUP_STEPS} steps, LR start factor: {INITIAL_LR_FACTOR}")

    # Model, opt, scheduler
    model = SimpleUNetClassifier(target_names=all_targets, num_binary=3).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)

    best_val_metric = -np.inf
    global_step = 0

    for epoch in range(1, N_EPOCHS + 1):
        print(f'\n--- Fold {fold} | Epoch {epoch}/{N_EPOCHS} ---')
        train_loss, train_f1, global_step = run_epoch(
            train_loader, model, criterion_dict, optimizer=optimizer,
            accum_steps=ACCUM_STEPS, train=True, device=DEVICE,
            global_step=global_step, warmup_steps=WARMUP_STEPS,
            base_lr=BASE_LR, initial_lr_factor=INITIAL_LR_FACTOR,
            df_targets=df_targets
        )
        val_loss, val_f1, _ = run_epoch(
            val_loader, model, criterion_dict, train=False, device=DEVICE,
            df_targets=df_targets
        )

        current_lr = optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, LR={current_lr:.6f}')
        print("Train Macro F1:", {k: f"{v:.4f}" for k, v in train_f1.items()})
        print("Val Macro F1:", {k: f"{v:.4f}" for k, v in val_f1.items()})

        scheduler_metric = val_f1.get('grade_macro_f1', 0.0)
        if epoch >= WARMUP_EPOCHS:
            scheduler.step(scheduler_metric)
        else:
            print(f"Warmup phase (Epoch {epoch}/{WARMUP_EPOCHS}). LR controlled manually. Current LR: {current_lr:.6f}")

        if scheduler_metric > best_val_metric:
            print(f"Validation F1 ('grade') improved ({best_val_metric:.4f} --> {scheduler_metric:.4f}). Saving model...")
            best_val_metric = scheduler_metric
            torch.save(model.state_dict(), f'best_fold{fold}_epoch{epoch}.pth')

    fold_results.append(best_val_metric)

print('\n===== CV RESULTS (Best Validation grade_macro_f1 per fold) =====')
for i, v in enumerate(fold_results, 1):
    print(f'Fold {i}: {v:.4f}')
print(f'Mean best val F1: {np.mean(fold_results):.4f} ± {np.std(fold_results):.4f}')


==========  FOLD 1/5  (96 train / 24 val) ==========
Warmup: 5 epochs, 60 steps, LR start factor: 0.01

--- Fold 1 | Epoch 1/30 ---


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


train:   0%|          | 0/192 [00:00<?, ?it/s]


Epoch train finished. Calculating final metrics...
--- IDH1 ---
  Macro F1: 0.3379
  Accuracy: 0.5104
  Targets: [94 98]
  Preds:   [  0 192]
--------------------
--- MGMT ---
  Macro F1: 0.3786
  Accuracy: 0.6094
  Targets: [117  75]
  Preds:   [192]
--------------------
--- 1p/19q ---
  Macro F1: 0.4861
  Accuracy: 0.6146
  Targets: [133  59]
  Preds:   [155  37]
--------------------
--- grade ---
  Macro F1: 0.2199
  Accuracy: 0.3542
  Targets: [27 75 90]
  Preds:   [ 18 172   2]
--------------------


valid:   0%|          | 0/24 [00:00<?, ?it/s]

...


Epoch valid finished. Calculating final metrics...
--- IDH1 ---
  Macro F1: 0.5000
  Accuracy: 0.5000
  Targets: [12 12]
  Preds:   [12 12]
--------------------
--- MGMT ---
  Macro F1: 0.2500
  Accuracy: 0.3333
  Targets: [16  8]
  Preds:   [ 0 24]
--------------------
--- 1p/19q ---
  Macro F1: 0.4080
  Accuracy: 0.4583
  Targets: [20  4]
  Preds:   [11 13]
--------------------
--- grade ---
  Macro F1: 0.3815
  Accuracy: 0.3750
  Targets: [ 3  8 13]
  Preds:   [ 2 14  8]
--------------------
Epoch 29: train_loss=1.6640, val_loss=3.3610, LR=0.000037
Train Macro F1: {'IDH1_macro_f1': '0.6085', 'MGMT_macro_f1': '0.4003', '1p/19q_macro_f1': '0.5674', 'grade_macro_f1': '0.6596'}
Val Macro F1: {'IDH1_macro_f1': '0.5000', 'MGMT_macro_f1': '0.2500', '1p/19q_macro_f1': '0.4080', 'grade_macro_f1': '0.3815'}

--- Fold 5 | Epoch 30/30 ---


train:   0%|          | 0/192 [00:00<?, ?it/s]


Epoch train finished. Calculating final metrics...
--- IDH1 ---
  Macro F1: 0.6719
  Accuracy: 0.6719
  Targets: [99 93]
  Preds:   [ 92 100]
--------------------
--- MGMT ---
  Macro F1: 0.4722
  Accuracy: 0.4792
  Targets: [115  77]
  Preds:   [ 55 137]
--------------------
--- 1p/19q ---
  Macro F1: 0.6112
  Accuracy: 0.6146
  Targets: [133  59]
  Preds:   [ 77 115]
--------------------
--- grade ---
  Macro F1: 0.7012
  Accuracy: 0.6406
  Targets: [ 22  66 104]
  Preds:   [ 27 100  65]
--------------------


valid:   0%|          | 0/24 [00:00<?, ?it/s]


Epoch valid finished. Calculating final metrics...
--- IDH1 ---
  Macro F1: 0.4857
  Accuracy: 0.5000
  Targets: [12 12]
  Preds:   [ 8 16]
--------------------
--- MGMT ---
  Macro F1: 0.6243
  Accuracy: 0.6250
  Targets: [16  8]
  Preds:   [ 7 17]
--------------------
--- 1p/19q ---
  Macro F1: 0.2804
  Accuracy: 0.2917
  Targets: [20  4]
  Preds:   [ 7 17]
--------------------
--- grade ---
  Macro F1: 0.3217
  Accuracy: 0.3333
  Targets: [ 3  8 13]
  Preds:   [ 4 14  6]
--------------------
Epoch 30: train_loss=1.5070, val_loss=3.0761, LR=0.000037
Train Macro F1: {'IDH1_macro_f1': '0.6719', 'MGMT_macro_f1': '0.4722', '1p/19q_macro_f1': '0.6112', 'grade_macro_f1': '0.7012'}
Val Macro F1: {'IDH1_macro_f1': '0.4857', 'MGMT_macro_f1': '0.6243', '1p/19q_macro_f1': '0.2804', 'grade_macro_f1': '0.3217'}

===== CV RESULTS (Best Validation grade_macro_f1 per fold) =====
Fold 1: 0.4093
Fold 2: 0.4823
Fold 3: 0.5510
Fold 4: 0.3387
Fold 5: 0.4478
Mean best val F1: 0.4458 ± 0.0710
